In [1]:
import PIL
import numpy as np
import torch
from torchvision.transforms.v2 import Compose, GaussianBlur, RandomEqualize, RandomSolarize, RandomApply

import wandb
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset
from tasks.SRDiffTrainer import SRDiffTrainer
from tasks.SR3Trainer import SR3Trainer
from utils.model_utils import load_model

- Generar imagenes bicubicas
- Construir Dataset
- Construir Dataloader
- SR3
- SRdiff
- SR3+

## 64 -> 256

## Entrenamiento

In [2]:
lr_size = 64
hr_size = 256
batch_size = 16
dataset_dir = 'E:\\TFG\\air_dataset'

In [3]:
transforms = Compose(
    [RandomApply(transforms= [GaussianBlur(7)], p = 0.5),
    RandomEqualize()]
)

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation = transforms)
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### SRDiff

In [4]:
from models.SRDiff.diffusion import GaussianDiffusion
from models.SRDiff.diffsr_modules import Unet, RRDBNet
losstype = "l2"
model_name = f"SRDiff{losstype}DA"
hidden_size = 64
dim_mults = [1,2,2,4]
rrdb_num_features = 32
rrdb_num_blocks = 8
timesteps = 100

denoise_fn = Unet(
    hidden_size, out_dim=3, cond_dim=rrdb_num_features, dim_mults=dim_mults, rrdb_num_block=rrdb_num_blocks, sr_scale=4)

rrdb = RRDBNet(3, 3, rrdb_num_features, rrdb_num_blocks, rrdb_num_features// 2)

model = GaussianDiffusion(
    denoise_fn=denoise_fn,
    rrdb_net=rrdb,
    timesteps= timesteps,
    loss_type=losstype
)
#model = load_model(model, f"SRDiff{losstype}.pt", "models_state_dic")
model.to(device)  

lr= 0.0002
decay_steps= 100000
gamma = 0.5

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=decay_steps, gamma=gamma)
max_steps = 30
hyperparams = {
    "max_steps": 100,
    "model": "SRDiff",
    "learning_rate": lr,
    "decay_steps": decay_steps,
    "gamma": gamma,
    "batch_size": batch_size,
    "hidden_size": hidden_size,
    "dim_mults": dim_mults,
    "rrdb_num_features": rrdb_num_features,
    "rrdb_num_blocks": rrdb_num_blocks,
    "loss_type": losstype,
    "epochs": max_steps
}
project_name = "SR model benchmarking"
run_name = "SRDiff with data augmentation"
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SRDiffTrainer(metrics_used=["ssim", "psnr"], model_name=model_name)
trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(max_steps):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    train_loss = trainer.train(train_dataloader)
    torch.cuda.empty_cache()
    trainer.save_model("models_state_dic")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()   

wandb: Currently logged in as: adrianpereramoreno (ladoscuro). Use `wandb login --relogin` to force relogin


100%|██████████| 50/50 [12:47<00:00, 15.36s/batch, n_samples=1, psnr=5.78, ssim=0.0329]


psnr,▁
ssim,▁
train_loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,5.5199
ssim,0.08398
train_loss,0.03607
validation_loss,0.03739


 ### SR3

In [5]:
from models.SR3.diffusion import GaussianDiffusion
from models.SR3.model import UNet
hyperparams = {
    "steps" : 2000,
    "sample_steps" : 100,
    "lr":0.0002,
    "epochs":50,
    "eta_min":1e-7,
    "decay_steps": 100000,
    "gamma" : 0.5,  
    "model" : "SR3"
}
model = UNet(3, hyperparams["steps"]) #Valores por defecto ya que la tarea base es la misma upsample por 4
SR3_model = GaussianDiffusion(model, hyperparams["steps"], hyperparams["sample_steps"])
SR3_model.to(device)

GaussianDiffusion(
  (model): UNet(
    (emb): GammaEmbedding(
      (linear1): Linear(
        (linear): Linear(in_features=3, out_features=12, bias=True)
      )
      (silu): SiLU()
      (linear2): Linear(
        (linear): Linear(in_features=12, out_features=12, bias=True)
      )
    )
    (conv1): Conv2d(
      (conv): Conv2d(6, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (down): ModuleList(
      (0-2): 3 x WideResNetBlock(
        (gn1): GroupNorm(
          (group_norm): GroupNorm(3, 3, eps=1e-05, affine=True)
        )
        (silu1): SiLU()
        (conv1): Conv2d(
          (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (silu2): SiLU()
        (linear1): Linear(
          (linear): Linear(in_features=12, out_features=3, bias=True)
        )
        (gn2): GroupNorm(
          (group_norm): GroupNorm(3, 3, eps=1e-05, affine=True)
        )
        (silu3): SiLU()
        (dropout): Dropout(p=0.0, inplace=False)

In [6]:
optimizer = torch.optim.Adam(SR3_model.parameters(), lr=hyperparams["lr"])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["gamma"])

In [7]:
project_name = "SR model benchmarking"
run_name = "SR3 with data aug"
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SR3Trainer(metrics_used=["ssim", "psnr"], model_name="SR3")
trainer.set_model(SR3_model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    
    train_loss = trainer.train(train_dataloader)
    torch.cuda.empty_cache()
    print(train_loss)
    trainer.save_model("models_state_dic")

    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()   

100%|██████████| 148/148 [00:21<00:00,  7.01batch/s]


tensor(0.7691, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 148/148 [00:20<00:00,  7.14batch/s]


tensor(0.7041, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 148/148 [00:20<00:00,  7.17batch/s]


...

tensor(0.1002, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 148/148 [00:20<00:00,  7.08batch/s]


tensor(0.1028, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 50/50 [02:37<00:00,  3.15s/batch]


psnr,▁
ssim,▁
train_loss,█▇▇▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▇▇▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,8.97627
ssim,0.02253
train_loss,0.10276
validation_loss,0.11252


#### SR3+

In [8]:
from models.SR3plus.diffusion import GaussianDiffusion
from models.SR3plus.model import UNet
hyperparams = {
    "steps" : 8000,
    "sample_steps" : 100,
    "lr":0.0002,
    "epochs":50,
    "eta_min":1e-7,
    "decay_steps": 100000,
    "gamma" : 0.5,
    "model" : "SR3+"
}
model = UNet(3, hyperparams["steps"], channel_expansions= [1, 2, 4, 4, 4, 8, 8, 8])
SR3plus_model = GaussianDiffusion(model, hyperparams["steps"], hyperparams["sample_steps"])
SR3plus_model.to(device)

GaussianDiffusion(
  (model): UNet(
    (emb): GammaEmbedding(
      (linear1): Linear(
        (linear): Linear(in_features=3, out_features=12, bias=True)
      )
      (silu): SiLU()
      (linear2): Linear(
        (linear): Linear(in_features=12, out_features=12, bias=True)
      )
    )
    (conv1): Conv2d(
      (conv): Conv2d(6, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (down): ModuleList(
      (0-2): 3 x WideResNetBlock(
        (gn1): GroupNorm(
          (group_norm): GroupNorm(3, 3, eps=1e-05, affine=True)
        )
        (silu1): SiLU()
        (conv1): Conv2d(
          (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (silu2): SiLU()
        (linear1): Linear(
          (linear): Linear(in_features=12, out_features=3, bias=True)
        )
        (gn2): GroupNorm(
          (group_norm): GroupNorm(3, 3, eps=1e-05, affine=True)
        )
        (silu3): SiLU()
        (dropout): Dropout(p=0.0, inplace=False)

In [9]:
optimizer = torch.optim.Adam(SR3plus_model.parameters(), lr=hyperparams["lr"])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["gamma"])

In [10]:
 project_name = "SR model benchmarking"
run_name = "SR3+ with data aug"
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SR3Trainer(metrics_used=["ssim", "psnr"], model_name="SR3+")
trainer.set_model(SR3plus_model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    train_loss = trainer.train(train_dataloader)
    torch.cuda.empty_cache()
    trainer.save_model("models_state_dic") 

    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()   

100%|██████████| 50/50 [02:52<00:00,  3.45s/batch]


psnr,▁
ssim,▁
train_loss,█▇▇▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,7.47231
ssim,0.02284
train_loss,0.06443
validation_loss,0.06818
